# Соколов Никита ИУ6-54Б

In [ ]:
surname = "Соколов"

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  3
Задача № 2:  2


In [ ]:
import os
import math
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, avg, count, lit, pow, sum, collect_list, struct, when, expr, sqrt, broadcast, abs
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk'
os.environ['PATH'] = f'{os.environ["JAVA_HOME"]}/bin:{os.environ["PATH"]}'
os.environ['_JAVA_OPTIONS'] = '-Dhadoop.security.authentication=simple -Duser.name=spark'

spark = SparkSession.builder \
    .appName('hw4_variant_3_2') \
    .master('local[*]') \
    .getOrCreate()
    
spark.sparkContext.setLogLevel("WARN")
print("Spark Session Created")

Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/08 17:26:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session Created


In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, row_number

GENRES_V3 = ['Thriller', 'Sci-Fi', 'Adventure']

df_mov = spark.read.csv('/home/beekfin/Downloads/ml-latest-small/movies.csv', header=True, inferSchema=True)
df_rat = spark.read.csv('/home/beekfin/Downloads/ml-latest-small/ratings.csv', header=True, inferSchema=True)

df_exploded = df_mov.withColumn('single_genre', explode(split(col('genres'), '\\|'))) \
                    .filter(col('single_genre').isin(GENRES_V3))

movie_aggs = df_rat.groupBy('movieId').agg(
    count(col('rating')).alias('n_votes'),
    avg(col('rating')).alias('avg_rat')
)

df_full = df_exploded.join(movie_aggs, 'movieId')

def show_top_per_genre(dataframe, order_col, ascending=False, limit=10, filter_cond=None):
    sort_dir = col(order_col).asc() if ascending else col(order_col).desc()
    
    w = Window.partitionBy('single_genre').orderBy(sort_dir)
    
    tmp_df = dataframe
    if filter_cond is not None:
        tmp_df = tmp_df.filter(filter_cond)
        
    tmp_df.withColumn('rn', row_number().over(w)) \
          .filter(col('rn') <= limit) \
          .select('single_genre', 'title', 'n_votes', 'avg_rat') \
          .orderBy('single_genre', sort_dir) \
          .show(truncate=False)

print("1.1. Статистика по жанрам:")
df_exploded.groupBy('single_genre').count().show()

print("1.2. Топ популярных (по кол-ву голосов):")
show_top_per_genre(df_full, 'n_votes')

print("1.3. Топ популярных (голосов > 10, сортировка по возрастанию):")
show_top_per_genre(df_full, 'n_votes', ascending=True, filter_cond=(col('n_votes') > 10))

print("1.4. Топ рейтинг (High Rating, голосов >= 10):")
show_top_per_genre(df_full, 'avg_rat', filter_cond=(col('n_votes') >= 10))

print("1.5. Анти-топ рейтинг (Low Rating, голосов >= 10):")
show_top_per_genre(df_full, 'avg_rat', ascending=True, filter_cond=(col('n_votes') >= 10))

1.1. Статистика по жанрам:
+------------+-----+
|single_genre|count|
+------------+-----+
|    Thriller| 1894|
|   Adventure| 1263|
|      Sci-Fi|  980|
+------------+-----+

1.2. Топ популярных (по кол-ву голосов):
+------------+------------------------------------------------------------------------------+-------+------------------+
|single_genre|title                                                                         |n_votes|avg_rat           |
+------------+------------------------------------------------------------------------------+-------+------------------+
|Adventure   |Star Wars: Episode IV - A New Hope (1977)                                     |251    |4.231075697211155 |
|Adventure   |Jurassic Park (1993)                                                          |238    |3.75              |
|Adventure   |Toy Story (1995)                                                              |215    |3.9209302325581397|
|Adventure   |Star Wars: Episode V - The Empire Strikes Ba

In [ ]:
ds_train, ds_test = df_rat.randomSplit([0.8, 0.2], seed=999) # Другой сид

g_mean = ds_train.select(avg('rating')).first()[0]
print(f"Global Mean: {g_mean:.3f}")

baseline_prediction = ds_test.withColumn("base_pred", lit(g_mean))
evaluator_base = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="base_pred")
rmse_base = evaluator_base.evaluate(baseline_prediction)
print(f"Baseline RMSE: {rmse_base:.3f}")

def run_item_based_cf(training_set, testing_set, k=10):
    t_train = training_set.select(col('movieId').alias('u'), col('userId').alias('i'), 'rating')
    t_test  = testing_set.select(col('movieId').alias('u'),  col('userId').alias('i'), 'rating')

    means = t_train.groupBy('u').agg(avg('rating').alias('mu'))
    
    train_c = t_train.join(means, 'u').withColumn('r_dev', col('rating') - col('mu')).select('u', 'i', 'r_dev')
    
    pairs = train_c.alias('A').join(
        train_c.alias('B'),
        (col('A.i') == col('B.i')) & (col('A.u') < col('B.u'))
    )
    
    sim_agg = pairs.groupBy(col('A.u').alias('m1'), col('B.u').alias('m2')).agg(
        sum(col('A.r_dev') * col('B.r_dev')).alias('dot'),
        sum(pow(col('A.r_dev'), 2)).alias('sq1'),
        sum(pow(col('B.r_dev'), 2)).alias('sq2'),
        count('*').alias('cnt')
    ).filter(col('cnt') >= 3)

    sim_agg = sim_agg.withColumn('denom', sqrt(col('sq1')) * sqrt(col('sq2')))
    
    sim_matrix = sim_agg.withColumn('sim', 
                                    when(col('denom') == 0, 0.0)
                                    .otherwise(col('dot') / col('denom'))) \
                        .select('m1', 'm2', 'sim')
                        
    sim_full = sim_matrix.union(sim_matrix.select(col('m2').alias('m1'), col('m1').alias('m2'), 'sim'))
    
    w_sim = Window.partitionBy('m1').orderBy(col('sim').desc())
    
    neighbors = sim_full.withColumn('rank', row_number().over(w_sim)) \
                        .filter(col('rank') <= k) \
                        .select(col('m1').alias('target_movie'), col('m2').alias('neigh_movie'), 'sim')
                        
        
    pred_df = t_test.alias('T') \
        .join(neighbors.alias('N'), col('T.u') == col('N.target_movie')) \
        .join(t_train.alias('H'), (col('N.neigh_movie') == col('H.u')) & (col('T.i') == col('H.i'))) \
        .groupBy('T.u', 'T.i') \
        .agg(
            sum(col('N.sim') * col('H.rating')).alias('num'),
            sum(abs(col('N.sim'))).alias('den')
        ) \
        .withColumn('pred_val', when(col('den') == 0, lit(None)).otherwise(col('num') / col('den'))) \
        .select(col('T.u').alias('movieId'), col('T.i').alias('userId'), 'pred_val')
    
    return pred_df

pred_ib = run_item_based_cf(ds_train, ds_test, k=15) # K=15

final_pred_ib = ds_test.join(pred_ib, ['movieId', 'userId'], 'left_outer') \
                       .withColumn('prediction', 
                                   when(col('pred_val').isNotNull(), col('pred_val'))
                                   .otherwise(lit(g_mean))) # Fallback to Global Mean

evaluator_cf = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
metric_cf = evaluator_cf.evaluate(final_pred_ib)
print(f"Item-Based CF RMSE: {metric_cf:.3f}")

Global Mean: 3.500
Baseline RMSE: 1.039


Item-Based CF RMSE: 1.151


In [ ]:
from pyspark.ml.tuning import TrainValidationSplit

print("Настройка ALS модели...")

als = ALS(
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating", 
    coldStartStrategy="drop",
    nonnegative=True,
    seed=12345
)

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 15]) \
    .addGrid(als.regParam, [0.01, 0.1, 1.0]) \
    .build()

tvs = TrainValidationSplit(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator_cf, 
    trainRatio=0.8, 
    seed=12345
)

print("Start fit...")
tvs_model = tvs.fit(ds_train)

best_als = tvs_model.bestModel
print(f"Best Rank: {best_als.rank}")
print(f"Best RegParam: {best_als._java_obj.parent().getRegParam()}")

pred_als = best_als.transform(ds_test)
metric_als = evaluator_cf.evaluate(pred_als)
print(f"ALS RMSE: {metric_als:.3f}")

print("\n" + "="*30)
print(f"RESULTS COMPARISON")
print("="*30)
print(f"CF (Item-Based): {metric_cf:.4f}")
print(f"ALS (Matrix Fac): {metric_als:.4f}")

if metric_als < metric_cf:
    print(">> ALS победил (меньшая ошибка).")
else:
    print(">> CF победил (меньшая ошибка).")

Настройка ALS модели...
Start fit...
Best Rank: 15
Best RegParam: 0.1
ALS RMSE: 0.873

RESULTS COMPARISON
CF (Item-Based): 1.1514
ALS (Matrix Fac): 0.8734
>> ALS победил (меньшая ошибка).
